
# Table of Contents



Initialize Python session with imports.



In [1]:
from bitarray import bitarray
import numpy as np
from math import ceil, factorial, log, log10, sqrt
import timeit

## 21 Amicable numbers



### Q:



Let d(n) be defined as the sum of proper divisors of n (numbers less than n
which divide evenly into n).

If d(a) = b and d(b) = a, where a ≠ b, then a and b are an amicable pair and
each of a and b are called amicable numbers.

For example, the proper divisors of 220 are 1, 2, 4, 5, 10, 11, 20, 22, 44, 55
and 110; therefore d(220) = 284. The proper divisors of 284 are 1, 2, 4, 71 and
142; so d(284) = 220.

Evaluate the sum of all the amicable numbers under 10000.



### A:



This is actually a fairly straightforward problem if we have some way to look up
the sum of divisors for any number less than our upper limit. Firstly, we need
some way to sieve for the sum of divisors. The follow function accomplishes this
by iterating over all the divisors, adding itself and its quotient at all the
multiples.



In [2]:
def sum_divisors_sieve(n,proper=True):
    """Sieve for the sum of proper divisors for natural numbers less than ``n``.
    Optionally, we can include the number itself with ``proper=False``."""
    dsum = np.ones(n,dtype=int)
    dsum[:2] = 0
    for i in range(2,int((n-1)**0.5)+1):
        dsum[i*i] += i
        dsum[i*(i+1)::i] += i + np.arange(i+1,(n-1)//i+1)
    if not proper:
        dsum += np.arange(0,n)
    return dsum

assert(sum_divisors_sieve(10,True ) == [0,0,1,1,3,1,6,1,7,4]).all()
assert(sum_divisors_sieve(10,False) == [0,1,3,4,7,6,12,8,15,13]).all()

Now we can use this function to find all the amicable pairs under the chosen
limit. Since the array `dsum[a]` is analogous to the $d(a)$ function in the
problem statement, let's define the amicable pair `(b,a)` as

1.  `dsum(a)=b`
2.  `dsum(b)=a`
3.  `a>b`

Using this definition, we make sure to avoid counting perfect numbers, and make
sure that every pair we return is unique. The function returns a list of tuples.
I realize this is awkward to sum up later, but I felt it followed the
mathematics more closely to do it that way.



In [3]:
def amicable_pairs(n):
    """Find all the amicable pairs less than ``n``."""
    dsum = sum_divisors_sieve(n)
    pairs = []
    for a in range(1,n):
        if a>dsum[a]:
            if dsum[dsum[a]]==a:
                pairs += [(dsum[a],a)]
    return pairs

assert amicable_pairs(284)==[]
assert amicable_pairs(285)[0]==(220,284)

We finally have to sum up all the numbers in the list of tuples.



In [4]:
print(sum([sum(pair) for pair in amicable_pairs(10000)]))
%timeit sum([sum(pair) for pair in amicable_pairs(10000)])

31626
4.58 ms ± 474 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 22 Names scores



### Q:



Using names.txt (right click and 'Save Link/Target As&#x2026;'), a 46K text file
containing over five-thousand first names, begin by sorting it into alphabetical
order. Then working out the alphabetical value for each name, multiply this
value by its alphabetical position in the list to obtain a name score.

For example, when the list is sorted into alphabetical order, COLIN, which is
worth 3 + 15 + 12 + 9 + 14 = 53, is the 938th name in the list. So, COLIN would
obtain a score of 938 × 53 = 49714.

What is the total of all the name scores in the file?



### A:



I hate text problems. We'll read this file into a `numpy.array` of strings.



In [5]:
def read_words(filename):
    """Read a list of words from file."""
    with open(filename) as file:
        words = []
        for line in file:
            words += line.replace("\"", "").split(",")
    return np.array(words)

We need a way to find the alphabetical value of a word or list of words



In [6]:
def alphabetical_value(word):
    """Get the alphabetical value for a word or list of words."""
    if isinstance(word, str):
        word = [word]
    values = np.zeros(len(word), dtype=int)
    for i, word_i in enumerate(word):
        values[i] = sum([(ord(c)-64)%32 for c in word_i])
    return values

assert alphabetical_value("Bob") == 19
assert (alphabetical_value(["Bob", "Alice"]) == [19, 30]).all()

Since our names are in a `numpy.array`, sorting is trivial. Given that, multiply
their alphabetical value by their index and print the total.



In [7]:
names = np.sort(read_words("022.txt"))
print(sum(np.multiply(alphabetical_value(names),np.arange(1,len(names)+1))))

%timeit names = np.sort(read_words("022.txt")); sum(np.multiply(alphabetical_value(names),np.arange(1,len(names)+1)))

871198282
7.45 ms ± 718 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 23 Non-abundant sums



### Q:



A perfect number is a number for which the sum of its proper divisors is exactly
equal to the number. For example, the sum of the proper divisors of 28 would be
1 + 2 + 4 + 7 + 14 = 28, which means that 28 is a perfect number.

A number n is called deficient if the sum of its proper divisors is less than n
and it is called abundant if this sum exceeds n.

As 12 is the smallest abundant number, 1 + 2 + 3 + 4 + 6 = 16, the smallest
number that can be written as the sum of two abundant numbers is 24. By
mathematical analysis, it can be shown that all integers greater than 28123 can
be written as the sum of two abundant numbers. However, this upper limit cannot
be reduced any further by analysis even though it is known that the greatest
number that cannot be expressed as the sum of two abundant numbers is less than
this limit.

Find the sum of all the positive integers which cannot be written as the sum of
two abundant numbers.



### A:



This problem stands out among the Euler problems because it does not scale. In
fact, it can't scale. It says so in the problem statement. "All integers greater
than 28123 can be written as the sum of two abundant numbers." Given this, we
have no real need to make the problem scalable, but we can make it fast. Let's
start by finding all the abundant numbers, making sure to employ the sum of
divisors sieve from problem 21. We can also use a similar strategy for the
deficient numbers and perfect numbers if we so desire, but that is beyond the
scope of the problem at hand.



In [8]:
def abundant_numbers(n):
    """Find all the abundant numbers less than ``n``."""
    dsum = sum_divisors_sieve(n)
    return np.array([i for i, d in enumerate(dsum) if i<d])

assert len(abundant_numbers(12))==0
assert abundant_numbers(13)==[12]
assert(abundant_numbers(20)==[12,18]).all()

Now that we have the abundant numbers, we simply need to sieve over the abundant
numbers. Now, according to [Wolfram Alpha](http://mathworld.wolfram.com/AbundantNumber.html), the last number that we are searching
for is actually 20161, so I'll cap our search at 20162. I reused the caching
strategy from problem 5 to save memory, this time using a `numpy.array` of
`np.bool`. For each abundant number $a<n$, I sliced our list of abundant
numbers for all $b<n-a$, and sliced the cache for all indices $a+b$, setting
the cache at those indices to `False`. Finally, I construct a `numpy.array`
which contains all `i` such that `cache[i]` is `True`.



In [9]:
def nonabundant_sums(n=20162):
    """Find all the positive integers less than ``n`` which cannot be written as
    the sum of two abundant numbers."""
    n = 20162 if n>20162 else n

    cache = np.ones(n,dtype=np.bool)
    cache[0] = False # 0 is not positive

    ab = abundant_numbers(n)
    for a in ab:
        cache[a+ab[ab<n-a]] = False

    return np.array([i for i in range(0,n) if cache[i]])

assert sum(nonabundant_sums(24))==276
assert sum(nonabundant_sums(25))==276
assert sum(nonabundant_sums(30))==411
assert sum(nonabundant_sums(31))==411

The final result is pretty fast. The most savings made are in the list slicing
techniques from the `sum_divisors_sieve` and `nonabundant_sums` functions.



In [10]:
print(sum(nonabundant_sums()))
%timeit sum(nonabundant_sums())

4179871
65.3 ms ± 4.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 24 Lexicographic permutations



### Q:



A permutation is an ordered arrangement of objects. For example, 3124 is one
possible permutation of the digits 1, 2, 3 and 4. If all of the permutations are
listed numerically or alphabetically, we call it lexicographic order. The
lexicographic permutations of 0, 1 and 2 are:

012   021   102   120   201   210

What is the millionth lexicographic permutation of the digits 0, 1, 2, 3, 4, 5,
6, 7, 8 and 9?



### A:



To solve this with brute force, we would iterate up to the $k$th permutation.
That's $O(k)$ in run time, which is $O(n!)$ for the worst case. However,
there's a combinatoric shortcut that lets us do this in $O(n)$ time. I could
easily iterate through one million permutations, but it's not very hard to
generalize with that shortcut. Consider the characters $a_0,a_1,...,a_{n-1}$.
We want the fastest way to find the $k$th lexicographic permutation of these
characters. First, consider how many permutations there are where $a_0$ is the
leading character. 

\begin{eqnarray*}
a_0 a_1 \ldots a_{n-2} a_{n-1} \\
a_0 a_1 \ldots a_{n-1} a_{n-2} \\
\ldots \\
a_0 a_{n-1} \ldots a_1 a_2 \\
a_0 a_{n-1} \ldots a_2 a_1
\end{eqnarray*}

With $a_0$ fixed, we have $n-1$ more characters to permute, which gives
$(n-1)!$ ways to order the remaining characters. If and only if
$k\leq(n-1)!$, we know that $a_0$ must come first in the permutation. We
also know the $(n-1)!+1$th permutation must begin with $a_1$ since it
follows lexicographically.

$$ p_{(n-1)!}   = a_0 a_{n-1} \ldots a_2 a_1 $$

$$ p_{(n-1)!+1} = a_1 a_0 a_2 a_3 \ldots a_{n-2} a_{n-1} $$

Again, there are $(n-1)!$ consecutive permutations where $a_1$ is the
leading character. In fact, for every $a_i,i < n$, there are $(n-1)!$
permutations where it is the leading character. This makes it very easy to
figure out what character is the leading character. Let $a_i$ be the leading
character for the $k$th permutation. We can find $i$ using the following
equation.

$$ i = \lfloor (k-1)/(n-1)! \rfloor $$

Let's test this on the example in the problem statement. What's the first
character in the 4th permutation of 0,1,2?

$$ i = \lfloor (4-1)/(3-1)! \rfloor = \lfloor 3/2 \rfloor = 1 $$

What's the first character in the 5th permutation of 0,1,2?

$$ i = \lfloor (5-1)/(3-1)! \rfloor = \lfloor 4/2 \rfloor = 2 $$

Now we can figure out the first character with relative ease, but what about the
remaining characters? Well, if we consider the first character fixed, we now
have a new problem with only $n-1$ characters. The problem is recursive. The
new $k$ to feed into the next iteration of the algorithm is found by the
remainder after dividing by $(n-1)!$.

$$ k'-1 \equiv (k-1)\mod(n-1)! $$

Recursion is a crime against humanity, so here is the algorithm in a while loop,
and tests for all the problem statement examples.



In [11]:
def get_kth_permutation(char_list,k):
    """Find the ``k``th lexicographic permutation of a list of characters,
    ``char_list``."""
    if k<1 or k>factorial(len(char_list)):
        return None
    perm_list = []
    while len(char_list)>1 and k>1:
        fact = factorial(len(char_list)-1)
        i = (k-1)//fact
        k = (k-1)%fact + 1
        perm_list += [char_list[i]]
        char_list.pop(i)
    perm_list += char_list
    return perm_list

assert get_kth_permutation([0,1,2],0) is None
assert get_kth_permutation([0,1,2],1) == [0,1,2]
assert get_kth_permutation([0,1,2],2) == [0,2,1]
assert get_kth_permutation([0,1,2],3) == [1,0,2]
assert get_kth_permutation([0,1,2],4) == [1,2,0]
assert get_kth_permutation([0,1,2],5) == [2,0,1]
assert get_kth_permutation([0,1,2],6) == [2,1,0]
assert get_kth_permutation([0,1,2],7) == None

The final code runs in $O(n)$, where $n$ is the number of characters.



In [12]:
print(get_kth_permutation(list(range(0,10)),1000000))
%timeit get_kth_permutation(list(range(0,10)),1000000)

[2, 7, 8, 3, 9, 1, 5, 4, 6, 0]
3.94 µs ± 133 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## 25 1000-digit Fibonacci number



### Q:



The Fibonacci sequence is defined by the recurrence relation:

$F_n = F_{n−1} + F_{n−2}$, where $F_1 = 1$ and $F_2 = 1$.

Hence the first 12 terms will be:

\begin{eqnarray*}
F_1 = 1 \\
F_2 = 1 \\
F_3 = 2 \\
F_4 = 3 \\
F_5 = 5 \\
F_6 = 8 \\
F_7 = 13 \\
F_8 = 21 \\
F_9 = 34 \\
F_{10} = 55 \\
F_{11} = 89 \\
F_{12} = 144 \\
\end{eqnarray*}

The 12th term, $F_{12}$, is the first term to contain three digits.

What is the index of the first term in the Fibonacci sequence to contain 1000
digits?



### A:



Consider the formula for the $n$th Fibonacci number.

$$ F_n = \frac{\varphi^n-(-\varphi)^{-n}}{\sqrt{5}} $$

where $\varphi=\frac{1+\sqrt{5}}{2}$. There's a useful simplification we can
make if we look closely at the second term and substitute
$-\varphi^{-1}=-0.618$.

\begin{eqnarray*}
F_n & = & \frac{\varphi^n}{\sqrt{5}} - \frac{(-\varphi^{-1})^n}{\sqrt{5}} \\
    & = & \frac{\varphi^n}{\sqrt{5}} - \frac{(-0.618)^n}{\sqrt{5}}
\end{eqnarray*}

Since $ \left| \frac{(-0.618)^n}{\sqrt{5}} \right| < \frac{1}{2} $ for all
$n\geq0$, we can eliminate the second term and round to the nearest integer
with either the nearest integer function or the floor function.

\begin{eqnarray*}
F_n & = & \left[ \frac{\varphi^n}{\sqrt{5}} \right] \\
    & = & \lfloor \frac{\varphi^n}{\sqrt{5}} + \frac{1}{2} \rfloor
\end{eqnarray*}

If we want a number above a lower bound $L$, then $F_n\geq L$.

\begin{eqnarray*}
\frac{\varphi^n}{\sqrt{5}} & \geq & L \\
                 \varphi^n & \geq & \sqrt{5}\cdot L \\
               n\ln\varphi & \geq & \frac{1}{2}\ln5+\ln L \\
                         n & \geq & \frac{\frac{1}{2}\ln5+\ln L}
                                         {\ln\varphi} \\
                         n & =    & \lceil \frac{\frac{1}{2}\ln5 + \ln L}
                                                {\ln\varphi} \rceil
\end{eqnarray*}



In [13]:
def fibonacci_above(limit):
    """Find the index of the Fibonacci not below ``limit``."""
    phi = (1+sqrt(5))/2
    return ceil((log(5)/2+log(limit)) / log(phi))

assert fibonacci_above(10)==7
assert fibonacci_above(100)==12

The final run time is $O(1)$.



In [14]:
print(fibonacci_above(10**999))
%timeit fibonacci_above(10**999)

4782
3.41 µs ± 36.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## 26 Reciprocal cycles



### Q:



A unit fraction contains 1 in the numerator. The decimal representation of the
unit fractions with denominators 2 to 10 are given:

1/2	= 	0.5
1/3	= 	0.(3)
1/4	= 	0.25
1/5	= 	0.2
1/6	= 	0.1(6)
1/7	= 	0.(142857)
1/8	= 	0.125
1/9	= 	0.(1)
1/10	= 	0.1

Where 0.1(6) means 0.166666&#x2026;, and has a 1-digit recurring cycle. It can be
seen that 1/7 has a 6-digit recurring cycle.

Find the value of d < 1000 for which 1/d contains the longest recurring cycle in
its decimal fraction part.



### A:



Let $1/d$ be a unit fraction that contains a repeating cycle
with a period $n$-digits long. Let's say we know $d$ but want to know what
and how long the cycle is.

\begin{eqnarray*}
       1/d & = & 0.(a_1a_2\ldots a_n) \\
    10^n/d & = & a_1a_2\ldots a_n.(a_1a_2\ldots a_n) \\
(10^n-1)/d & = & a_1a_2\ldots a_n
\end{eqnarray*}

We don't know $n$, but we know that $a_1a_2\ldots a_n$ must be an integer.
Therefore, we need to find the smallest $n$ such that $d|10^n-1$. This is
pretty straightforward. However, what if $1/d$ isn't repeating? Do we even
need to find those? In fact, we don't. Any repeating decimal that doesn't have
the above form won't be the longest, and $d$ is always prime. What we are
looking for are called the [full reptend primes](https://en.wikipedia.org/wiki/Full_reptend_prime). A full reptend prime $p$
produces a repeating decimal $1/p=0.(a_1a_2\ldots a_{p-1})$, which is the
longest possible repeating decimal sequence for any reciprocal, relative to its
size, of course.

So in order to find the longest repeating reciprocal, we need to sieve the
primes. I'll use the Sieve of Eratosthenes (see [problem 5](001.md)) to do so. Now to find
the longest repeating decimal, we simply must find the last full reptend prime
below 1000. Note the special case $n < 8$. Look back at the problem statement
if it isn't clear why we need to add that.



In [15]:
from euler import esieve

def last_frp(n,primes=None):
    """Find the last full reptend prime less than ``n``. Optionally takes a list
    of primes as an argument.
    """
    if n<8:
        return 3 if n>3 else None

    if primes is None:
        primes = esieve(n)

    for p in primes[::-1]:
        period = 1
        while (10**period-1)%p!=0:
            period += 1
        if p-1 == period:
            return p

assert last_frp(10)==7
assert last_frp(100)==97

ModuleNotFoundError: No module named 'euler'

The algorithmic complexity should be the same as the Sieve of Eratosthenes,
$O(n\log\log n)$.



In [ ]:
print(last_frp(1000))
%timeit last_frp(1000)

The complexity scales well to the higher upper bound given by [hackerrank](https://www.hackerrank.com/contests/projecteuler/challenges/euler026/problem).



In [ ]:
print(last_frp(10000))
%timeit last_frp(10000)

## 27 Quadratic primes



### Q:



Euler discovered the remarkable quadratic formula:

$$ n^2+n+41 $$

It turns out that the formula will produce 40 primes for the consecutive integer
values $0\leq n\leq39$. However, when $n=40$, $40^2+40+41=40(40+1)+41$ is
divisible by 41, and certainly when $n=41$,$41^2+41+41$ is clearly divisible
by 41.

The incredible formula $n^2-79n+1601$ was discovered, which produces 80 primes
for the consecutive values $0\leq n\leq79$. The product of the coefficients,
−79 and 1601, is −126479.

Considering quadratics of the form:

$n2+an+b$, where $|a| < 1000$ and $|b|\leq1000$

where $|n|$ is the modulus/absolute value of $n$
e.g. $|11|=11$ and $|−4|=4$

Find the product of the coefficients, $a$ and $b$, for the quadratic
expression that produces the maximum number of primes for consecutive values of
$n$, starting with $n=0$.



### A:



For $n=0$, we have

\begin{eqnarray*}
0^2 + a\cdot 0 + b & = & p \\
                 b & = & p
\end{eqnarray*}

where $p$ is prime. Since $b=p$, $b$ is also prime. For $n=1$ we have

\begin{eqnarray*}
1^2 + a\cdot 1 + b & = & p \\
         1 + a + b & = & p
\end{eqnarray*}

Since $b$ must be odd, $b+1$ must be even, and $a$ must be odd.
Additionally, $a > -b$. With our search space reduced, we simply must use
brute force search to find the answer.



In [16]:
from euler import esieve, is_prime

limit = 1000
primes = esieve(limit+1)
nmax = 0
for b in primes:
    for a in range(-b+2,limit,2):
        n = 1
        while is_prime(n*n + a*n + b):
            n += 1
        n += -1
        if n >= nmax:
            nmax = n
            p = (a,b)

print(nmax, p)
print(p[0]*p[1])

ModuleNotFoundError: No module named 'euler'

## 28 Number spiral diagonals



### Q:



Starting with the number 1 and moving to the right in a clockwise direction a 5
by 5 spiral is formed as follows:

    21 22 23 24 25
    20  7  8  9 10
    19  6  1  2 11
    18  5  4  3 12
    17 16 15 14 13

It can be verified that the sum of the numbers on the diagonals is 101.

What is the sum of the numbers on the diagonals in a 1001 by 1001 spiral formed
in the same way?



### A:



Notice how the top right corner of each minor square is the square of an odd
integer ($1^2,3^2,5^2,\ldots$) corresponding to the size of the square. So if
we consider an arbitrary $n\times n$ spiral matrix, we can write all of the
corner values in terms of the size.

$$\begin{bmatrix}
    c_2    & \dots  & c_1    \\
    \vdots & \ddots & \vdots \\
    c_3    & \dots  & c_4 
  \end{bmatrix} $$

\begin{eqnarray*}
c_1 & = & n^2 \\
c_2 & = & n^2 - (n-1) \\
c_3 & = & n^2 - 2(n-1) \\
c_4 & = & n^2 - 3(n-1) \\
C   & = & 4n^2 - 6n + 6
\end{eqnarray*}



In [17]:
n = 1001
count = 1
for i in range(3,n+1,2):
    count += 4*i*i - 6*i + 6
print(count)

669171001


## 29 Distinct powers



### Q:



Consider all integer combinations of $a^b$ for $2\leq a\leq5$ and
$2\leq b\leq5$:

$$
2^2=4, 2^3=8, 2^4=16, 2^5=32 \\
3^2=9, 3^3=27, 3^4=81, 3^5=243 \\
4^2=16, 4^3=64, 4^4=256, 4^5=1024 \\
5^2=25, 5^3=125, 5^4=625, 5^5=3125
$$

If they are then placed in numerical order, with any repeats removed, we get the
following sequence of 15 distinct terms:

4, 8, 9, 16, 25, 27, 32, 64, 81, 125, 243, 256, 625, 1024, 3125

How many distinct terms are in the sequence generated by $a^b$ for
$2\leq a\leq100$ and $2\leq b\leq100$?



### A:



Easy by brute force and a set.



In [18]:
powers = set()
for a in range(2,101):
    for b in range(2,101):
        powers.add(a**b)
print(len(powers))

9183


## 30 Digit fifth powers



### Q:



Surprisingly there are only three numbers that can be written as the sum of
fourth powers of their digits:

\begin{eqnarray*}
1634 = 1^4 + 6^4 + 3^4 + 4^4 \\
8208 = 8^4 + 2^4 + 0^4 + 8^4 \\
9474 = 9^4 + 4^4 + 7^4 + 4^4 \\
\end{eqnarray*}

As $1 = 1^4$ is not a sum it is not included.

The sum of these numbers is 1634 + 8208 + 9474 = 19316.

Find the sum of all the numbers that can be written as the sum of fifth powers
of their digits.



### A:



This won't be difficult once we know a range of candidate numbers. Let $N$ be
an $n$ digit number.

$$ 10^{n-1} \leq N < 10^n $$

And let the sum of the fifth powers of the digits be equal to $N$.

$$ N = d_1d_2\dots d_n = d_1^5 + d_2^5 + \dots + d_n^5 $$

A digit can only take values $0,1,\dots,9$ so that enforces another upper
bound.

$$ N \leq n(9^5) = 59049n $$

And we get a new range for $N$.

$$ 10^{n-1} \leq N \leq 59049n $$

Looking at a table of values for the bounds of $N$, it is clear that there
cannot be solutions above 354294. Thus, we have a ceiling to iterate to.

| n|lower|upper|
|---|---|---|
| 2|10|118098|
| 3|100|177147|
| 4|1000|236196|
| 5|10000|295245|
| 6|100000|354294|
| 7|1000000|413343|



In [20]:
def digit_power_sums(p):
    """Find all the numbers whose digits to the ``p``th power is that number."""
    N = []
    power = [i**p for i in range(0,10)]

    n = 2
    limit = n*power[-1]
    while 10**(n-1) < limit:
        n += 1
        limit = n*power[-1]

    for i in range(10,limit+1):
        if sum([power[int(d)] for d in str(i)]) == i:
            N += [i]

    return N

assert digit_power_sums(4) == [1634, 8208, 9474]

In [21]:
N = digit_power_sums(5)
print(N)
print(sum(N))
%timeit digit_power_sums(5)

[4150, 4151, 54748, 92727, 93084, 194979]
443839
520 ms ± 25.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
